# Reconstruction parcimonieuse ou compressive sensing

## I. Cadre général et première tentative.

Le but du TP est de montrer que des techniques d'optimisation pour le Machine Learning permettent de récupérer des signaux qui admettent dans une certaine base une représentation parcimonieuse, c'est à dire mettant en jeu un faible nombre de coefficients. 

On s'intéresse donc à la reconstruction d'un vecteur $x$ (ici une image codées en niveaux de gris) en utilisant les valeurs de combinaisons aléatoire de certains pixels. Les combinaisons sont représentées par une matrice $\Phi$. 

Une première approche consiste à résoudre le problème $$ \min_x \frac{1}{2} ||\Phi x-y||_2^2$$ pour retrouver l'image.

**Question 1 :** Commenter et compléter le code constituant le reste de la section I.


In [ ]:
#
# Chargement des librairies
#
import matplotlib.pyplot as plt
import numpy as np
from scipy import fftpack
from PIL import Image
#Lecture de l'image
img = Image.open("cameraman.tif")
A =1/255*np.array(img)
print(np.shape(A))
plt.imshow(A,'gray')
plt.show()


In [ ]:
# Extraction d'une sous-image contrastée
k    = 30
orig = 60
Aloc   = A[orig-1:orig+k-1,orig-1:orig+k-1]
x = np.reshape(Aloc,[k*k,])
n = np.shape(x)[0]
m   = 500 
Phi = np.random.randn(m,n)
plt.imshow(Aloc,'gray')
plt.show()



**Question 2 :** Résoudre le problème de moindres carrés $$\min_x || \Phi x -y ||^2_2,$$ et afficher le résultat (attention, l'image doit être en niveaux de gris). Que constate-t-on?

In [ ]:
y     = np.matmul(Phi,x)
y     = np.reshape(y,[np.shape(y)[0],1]) 
## compléter

## Résolution



#Affichage



xls_img = np.reshape(xls,[k,k])
plt.imshow(xls_img,'gray')
plt.show()

#### II. Compressive sensing


On désire utiliser à présent la transformation discrète en cosinus pour obtenir une représentaiton parcimonieuse d'une image. Cette transfomation bijective est obtenue avec la fonction `dct`, son inverse s'appelant `idct`.

** Question 3 :** A partir de la fonction IDCT de Julia, calculer la matrice  $\Theta = \Phi*\Psi$, avec $\Psi$ la matrice représentative de IDCT dans la base canonique.


In [ ]:
Theta = np.zeros([m,n])
for ii in range(n):
    ek = np.zeros([1,n])
    ek[:,ii] = 1
    psi = np.transpose(fftpack.idct(ek,norm='ortho'))
    Theta[:,ii] = np.reshape((np.matmul(Phi,psi)),[m])


### II.1. Utilisation d'un algorithme de sous-gradient

** Qestion 3 :** Soit $\lambda > 0$ (ici $\lambda=100$). Expliquer l'intérêt de 
 calculer $x_{sg}= \Psi z$ où $z$ résout $$\min_z \frac{1}{2} ||\Phi  \cdot \Psi  \cdot z -y||_2^2+ \lambda ||z||_1.$$ 
 
** Qestion 4 :** Justifier la convexité de la fonctionnelle et donner son sous-différentiel.

** Qestion 5 :** Compléter ci-dessous le code de sous-gradient projeté. Vous considèrerez les différents choix de pas $\alpha_i$ vus en cours, et trouverez des réglages permettant d'obtenir un bon rendu visuel. On pourra se rappeler du fait que la solution de norme minimale a une norme inférieure à dix.


In [ ]:
z = np.zeros([k**2,1])
i=0 
Lambda=1e2
choix_pas = 2 # 1 cas a, 2 cas b, 3 cas c
while (i <= 1000) :
    i = i + 1 
    ## compléter
    
   
print(np.linalg.norm(z))
print(np.linalg.norm(np.matmul(np.linalg.pinv(Theta),y)))

**Question 5 :** Expliquer le code suivant et conclure sur l'utilisation du sous-gradient

In [ ]:
def my_function(n,k,z):
    x2 = np.zeros([n,1])
    for ii  in range(n):
        ek = np.zeros([1,n])
        ek[:,ii] = 1
        psi = np.transpose(fftpack.idct(ek))
        x2 = x2+psi*z[ii]
        
    x2 = np.reshape(x2,[k,k])
    alpha = 1/(np.max(x2)-np.min(x2))
    x2 = alpha*(x2-np.min(x2))
    return x2

x2 = my_function(n,k,z)
plt.imshow(x2,'gray')
plt.show()

# II.2. Utilisation d'un algorithme de sous-gradient projeté


** Question 6 :**  Expliquer l'intérêt de 
 calculer $x_{sgp}= \Psi z$ où $z$ résout $$\min_{\Phi  \cdot \Psi  \cdot z =y}  ||z||_1.$$

** Question 7 :** Rappeler la formule de la projection sur le convexe $\{x, \Phi  \cdot \Psi  \cdot z =y \}$ vue en cours.

** Question 8 :** Résoudre ce problème par l'algorithme de sous-gradient projeté et finaliser le calcul de la solution.